<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Financial_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.3 MB/s eta 0:00:00


In [2]:
import openai
import os
from getpass import getpass
#
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [4]:
prompt = """
Please retrieve company name, revenue, net income and earnings per share (a.k.a. EPS)
from the following news article. If you can't find the information from this article
then return "". Do not make things up.
Then retrieve a stock symbol corresponding to that company. For this you can use
your general knowledge (it doesn't have to be from this article). Always return your
response as a valid JSON string. The format of that string should be this,
    {
        "Company Name": "Walmart",
        "Stock Symbol": "WMT",
        "Revenue": "12.34 million",
        "Net Income": "34.78 million",
        "EPS": "2.1 $"
    }
News Article:
==============
```
{text}

```

"""

In [5]:
text = input("Enter the Financial Data:")
prompt = prompt.replace("{text}",text)

Enter the Financial Data:Tesla's Earning news in text format: Tesla's earning this quarter blew all the estimates. They reported 4.5 billion $ profit against a revenue of 30 billion $. Their earnings per share was 2.3 $


In [6]:
prompt

'\nPlease retrieve company name, revenue, net income and earnings per share (a.k.a. EPS)\nfrom the following news article. If you can\'t find the information from this article \nthen return "". Do not make things up.    \nThen retrieve a stock symbol corresponding to that company. For this you can use\nyour general knowledge (it doesn\'t have to be from this article). Always return your\nresponse as a valid JSON string. The format of that string should be this, \n    {\n        "Company Name": "Walmart",\n        "Stock Symbol": "WMT",\n        "Revenue": "12.34 million",\n        "Net Income": "34.78 million",\n        "EPS": "2.1 $"\n    }\nNews Article:\n==============\n```\nTesla\'s Earning news in text format: Tesla\'s earning this quarter blew all the estimates. They reported 4.5 billion $ profit against a revenue of 30 billion $. Their earnings per share was 2.3 $\n\n```\n  \n'

In [9]:
openai.api_key = os.environ["OPENAI_API_KEY"]
response = openai.ChatCompletion.create(
                                model="gpt-3.5-turbo",
                                messages=[{"role": "user","content": prompt}]
                                )
content = response.choices[0]['message']['content']

In [10]:
content

'{\n    "Company Name": "Tesla",\n    "Stock Symbol": "TSLA",\n    "Revenue": "30 billion $",\n    "Net Income": "4.5 billion $",\n    "EPS": "2.3 $"\n}'

In [11]:
import pandas as pd
import json
try:
  data = json.loads(content)
  df=  pd.DataFrame(data.items(), columns=["Measure", "Value"])

except (json.JSONDecodeError, IndexError):
        pass

In [13]:
print(df.to_string())

        Measure          Value
0  Company Name          Tesla
1  Stock Symbol           TSLA
2       Revenue   30 billion $
3    Net Income  4.5 billion $
4           EPS          2.3 $
